In [11]:
import pandas as pd
import numpy as np

### First-Looking and Determining Problems

In [12]:
defence_companies = pd.read_csv("defence_companies_from_2005.csv")
df = defence_companies.copy()
df.head()

,Year,Rank,Company,Leadership,Country,Defense_Revenue_From_A_Year_Ago,Defense_Revenue_From_Two_Years_Ago,%Defense Revenue Change,Total Revenue,%of Revenue from Defence
0,2020,1,Lockheed Martin 1,"James D. Taiclet, President and CEO *",U.S.,"$56,606.00","$50,536.00",12%,"$59,812.00",95%
1,2020,2,Boeing,"David Calhoun, President and CEO *",U.S.,"$34,300.00","$34,050.00",1%,"$76,559.00",45%
2,2020,3,General Dynamics 2,"Phebe Novakovic, Chairman and CEO",U.S.,"$29,512.00","$27,507.00",7%,"$39,350.00",75%
3,2020,4,Northrop Grumman,"Kathy J. Warden, Chairman, CEO and President",U.S.,"$28,600.00","$25,300.00",13%,"$33,841.00",85%
4,2020,5,Raytheon Company 1 3,"Thomas Kennedy, Chairman and CEO",U.S.,"$27,448.00","$25,163.94",9%,"$29,200.00",94%


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 10 columns):
Year                                  1600 non-null int64
Rank                                  1600 non-null int64
Company                               1600 non-null object
Leadership                            1600 non-null object
Country                               1600 non-null object
Defense_Revenue_From_A_Year_Ago       1600 non-null object
Defense_Revenue_From_Two_Years_Ago    1582 non-null object
%Defense Revenue Change               1087 non-null object
Total Revenue                         1595 non-null object
%of Revenue from Defence              1595 non-null object
dtypes: int64(2), object(8)
memory usage: 125.1+ KB


<b>Interpretation of the Head and Info Outputs</b>
<hr>
<b>Company: </b>There are numbers with company names but they should not be there.<br>
<b>Country: </b>We should look its inside, is there a different short country name which mean are same<br>
<b>Defense_Revenue_From_A_Year_Ago: </b>We should change type and look its inside too. Is there different type from float.<br>
<b>Defense_Revenue_From_Two_Years_Ago: </b>Interpretation is same with "Defense_Revenue_From_A_Year_Ago"<br>
<b>%Defense Revenue Change: </b>There are a lot of null values which i expect. When we look the revenues, Both do not have null values as its. We can fill most of them thanks to both of revenues<br>
<b>Total Revenue: </b>Interpretation is same with "Defense_Revenue_From_A_Year_Ago"<br>
<b>%of Revenue from Defence : </b>Interpretation is same with "Defense_Revenue_From_A_Year_Ago"<br>




In [14]:
df.Country.unique()

array(['U.S.', 'China', 'U.K.', 'Netherlands/France', 'Italy', 'France',
       'Russia', 'Japan', 'Israel', 'South Korea', 'Germany', 'Sweden',
       'Netherlands', 'India', 'Turkey', 'Singapore', 'Australia',
       'Canada', 'Switzerland', 'Norway', 'Brazil', 'Spain', 'Belgium',
       'Finland', 'South Africa', 'Ukraine', 'US', 'UK', 'canada',
       'Swiss', 'So. Korea', 'So Africa', 'Ireland', 'Kuwait', 'Korea',
       'U.S', 'S. Africa'], dtype=object)

<b>There are more than one short country name which mean are same. We have to fix it.</b><br>
<b>Examples: </b><br>
"U.S.", "U.S", "US" -> USA<br>
"So. Korea", "South Korea", "Korea(This is not from North, You can check the companies)" -> South Korea<br>
"Swiss", "Switzerland" -> Switzerland<br>
"S. Africa", "So Africa" -> South Africa<br><br>
<b>There is a Upper-Lower Problem</b><br>
<b>Examples: </b><br>
"canada" -> Canada<br><br>
<b>There is a company which belongs to two countries(Netherlands/France): Airbus. We will fix it</b><br>
"Netherlands/France" -> Netherlands and France


### Fixing Company Column

In [15]:
a = df.Company.str.replace("\d","") # To Remove Numbers
a = a.str.replace("[^\w\s]","") # To Remove Punctuations
a = a.str.strip() # To Remove Space Characters

df["Company"] = a

### Fixing Country Column

In [16]:
df["Country"] = df["Country"].str.replace("So Africa","South Africa").replace("S. Africa","South Africa")
df["Country"] = df["Country"].str.replace("U.S.","USA").replace("US","USA").replace("U.S","USA")
df["Country"] = df["Country"].str.replace("U.K.","United Kingdom").replace("UK","United Kingdom")
df["Country"] = df["Country"].str.replace("So. Korea","South Korea").replace("Korea","South Korea")
df["Country"] = df["Country"].str.replace("canada","Canada").replace("Swiss","Switzerland")
b = df[df.Country == "Netherlands/France"]
b.Country = ["France"] * 4 
df = df.append(b, ignore_index=True)
df.Country = df.Country.apply(lambda x: x.split("/")[0] if "/" in x else x)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### Fixing Defense_Revenue_From_Last_Year Column

<b>The column includes such as 778.0.9 value, we should change it to 778.09 before doing float. I understood to be 778.09 from its upper and lower companies revenue.</b>

In [17]:
df["Defense_Revenue_From_A_Year_Ago"] = df["Defense_Revenue_From_A_Year_Ago"].str.replace(",","")
df["Defense_Revenue_From_A_Year_Ago"] = df["Defense_Revenue_From_A_Year_Ago"].str.replace("$","")
df.iloc[1072,5] = "778.09"
df["Defense_Revenue_From_A_Year_Ago"] = df["Defense_Revenue_From_A_Year_Ago"].astype(np.float32)


### Fixing Defense_Revenue_From_Two_Years_Ago Column

<b>The column includes such as "~", "-" and "NR" values</b>

In [18]:
df["Defense_Revenue_From_Two_Years_Ago"] = df["Defense_Revenue_From_Two_Years_Ago"].str.replace(",","")
df["Defense_Revenue_From_Two_Years_Ago"] = df["Defense_Revenue_From_Two_Years_Ago"].str.replace("$","")
df["Defense_Revenue_From_Two_Years_Ago"] = df["Defense_Revenue_From_Two_Years_Ago"].apply(lambda x: np.nan if (x=="~" or x=="-" or x=="NR") else x)
df["Defense_Revenue_From_Two_Years_Ago"] = df["Defense_Revenue_From_Two_Years_Ago"].astype(np.float32)


### Fixing Total Revenue Column

<b>The column includes such as "~", "-" and "NR" values</b>

In [19]:
df["Total Revenue"] = df["Total Revenue"].str.replace(",","")
df["Total Revenue"] = df["Total Revenue"].str.replace("$","")
df["Total Revenue"] = df["Total Revenue"].apply(lambda x: np.nan if (x=="~" or x=="-" or x=="NR") else x)
df["Total Revenue"] = df["Total Revenue"].astype(np.float32)

### Updating %Defense Revenue Change Column From Revenues Which One and Two Years Ago

In [20]:
a = ((df["Defense_Revenue_From_A_Year_Ago"]-df["Defense_Revenue_From_Two_Years_Ago"])/df["Defense_Revenue_From_Two_Years_Ago"])*100
df["%Defense Revenue Change"] = a

### Updating %of Revenue from Defence Column From Revenues Which Total and A Year Ago

In [21]:
df["%of Revenue from Defence"] = df["Defense_Revenue_From_A_Year_Ago"] / df["Total Revenue"] * 100

### Changing Float Columns Fraction View

In [22]:
pd.options.display.float_format = "{:,.2f}".format


### Last Status

In [23]:
df.head()

,Year,Rank,Company,Leadership,Country,Defense_Revenue_From_A_Year_Ago,Defense_Revenue_From_Two_Years_Ago,%Defense Revenue Change,Total Revenue,%of Revenue from Defence
0,2020,1,Lockheed Martin,"James D. Taiclet, President and CEO *",USA,"56,606.00","50,536.00",12.01,"59,812.00",94.64
1,2020,2,Boeing,"David Calhoun, President and CEO *",USA,"34,300.00","34,050.00",0.73,"76,559.00",44.80
2,2020,3,General Dynamics,"Phebe Novakovic, Chairman and CEO",USA,"29,512.00","27,507.00",7.29,"39,350.00",75.00
3,2020,4,Northrop Grumman,"Kathy J. Warden, Chairman, CEO and President",USA,"28,600.00","25,300.00",13.04,"33,841.00",84.51
4,2020,5,Raytheon Company,"Thomas Kennedy, Chairman and CEO",USA,"27,448.00","25,163.94",9.08,"29,200.00",94.00


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1604 entries, 0 to 1603
Data columns (total 10 columns):
Year                                  1604 non-null int64
Rank                                  1604 non-null int64
Company                               1604 non-null object
Leadership                            1604 non-null object
Country                               1604 non-null object
Defense_Revenue_From_A_Year_Ago       1604 non-null float32
Defense_Revenue_From_Two_Years_Ago    1555 non-null float32
%Defense Revenue Change               1555 non-null float32
Total Revenue                         1586 non-null float32
%of Revenue from Defence              1586 non-null float32
dtypes: float32(5), int64(2), object(3)
memory usage: 94.1+ KB


In [25]:
df.to_csv("defence_companies_from_2005_cleaned", index=False, header=True)